In [ ]:
# default_exp manual_opt

In [ ]:
# hide
from nbdev.showdoc import *

# manual_opt

> Functions enabling a manual optimization of signal significance for a phenomenological problem (primarily for the purposes of comparison with the performance of a machine learning algorithm)

In [ ]:
# export
import numpy as np
import itertools

In [ ]:
# export
def manual_significance(S,Bs,se,bes):
    """Returns signal significance given yield and efficiency for signal and background"""
    return (S * se) / np.sqrt(S * se + sum([B * be for B, be in zip(Bs, bes)]))

In [ ]:
# export
def points2hrs(points, points2sec=126/(5**3 * 6)):
    """Returns number of hours required to compute significance for given number of points"""
    return points * points2sec * 3600**-1

In [ ]:
# export
def maxhrs2points(maxhrs, points2sec=126/(5**3 * 6)):
    """Returns maximal number of points capable of being computed during a given number of hours"""
    return maxhrs * 3600 * points2sec**-1

In [ ]:
# export
def sig_grid(signal, backgrounds, numS, numBs, cuts):
    r"""
    Computes signal significance for a high-dimensional grid of event selection criteria.
    
    `signal` should contain 2D signal data, `backgrounds` should be a list where the $i$th element
    contains 2D background data for the $i$th background type, `numS` should give signal yield,
    `numBs` should be a list of background yields for each background type, and `cuts` should 
    take the following form.
    
    Each row of `cuts` should take the form [index, isCutBelow, vals]
    where `index` gives the index of the feature being cut on
    in the signal/background data, `isCutBelow` is a boolean specifying
    if that variable should involve removing data points below (True) or
    above (False) the given values, and `vals` is a list of values to cut at
    """
    
    indices = [row[0] for row in cuts]
    isCutBelows = [row[1] for row in cuts]
    points = list(itertools.product(*[row[-1] for row in cuts]))
    sigsize = len(signal)
    bgsizes = np.array([len(bg) for bg in backgrounds])
    sigs = []
    sigsizeprimes = []
    bgsizesprimes = []
    for point in points:
        sigsizeprime = np.count_nonzero(np.logical_and.reduce(
            np.array([np.where(signal[:,index] > val, isCutBelow, not isCutBelow) 
                      for index, isCutBelow, val in zip(indices, isCutBelows, point)])))
        bgsizesprime = np.array([np.count_nonzero(np.logical_and.reduce(
            np.array([np.where(background[:,index] > val, isCutBelow, not isCutBelow) 
                      for index, isCutBelow, val in zip(indices, isCutBelows, point)]))) for background in backgrounds])
        sigsizeprimes.append(sigsizeprime)
        bgsizesprimes.append(bgsizesprime)
        sigs.append(manual_significance(numS, numBs, sigsizeprime/sigsize, bgsizesprime/bgsizes))
    return [points, sigs, np.array(sigsizeprimes)/sigsize, np.array(bgsizesprimes)/bgsizes]

In [ ]:
# export
def opt_sig(signal, backgrounds, numS, numBs, cuts, num_iter=3):
    """
    Iteratively calls `sig_grid` with finer and finer spectra of event selection criteria
    (kinematic cuts) to check
    """
    for i in range(num_iter):
        points, sigs, _, __ = sig_grid(signal, backgrounds, numS, numBs, cuts)
        max_sig_pt = points[sigs.index(max(sigs))]
        for j, val in enumerate(max_sig_pt):
            index = np.where(cuts[j][-1]==val)[0][0]
            length = len(cuts[j][-1])
            min_index, max_index = [max(0, index - 1), min(length - 1, index + 1)]
            cuts[j][-1] = np.linspace(cuts[j][-1][min_index], cuts[j][-1][max_index], 5)
    return [points[sigs.index(max(sigs))], max(sigs)]